In [101]:
import pandas as pd
import openai
import numpy as np
import copy

In [76]:
# Set OpenAI API key
OPENAI_API_KEY = ""

In [77]:
client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [2]:
df = pd.read_parquet('/Users/navneet/git/research/between-silence-and-speech/data/all_data_output.parquet')
df.head()

,model_name,statement,language1,language2,prompt_language1,prompt_language2,prompt_baseline,response_language1,response_language2,response_baseline,...,translated_response_language1_refusal,translated_response_language2_polarity,translated_response_language2_subjectivity,translated_response_language2_refusal,response_baseline_polarity,response_baseline_subjectivity,response_baseline_refusal,translated_response_lang1_distance,translated_response_lang2_distance,response_baseline_distance
0,llama3-8b,The Plame affair was a political scandal in 20...,English,English,You are given a statement and must respond in ...,You are given a statement and must respond in ...,You are given a statement and must respond in ...,The disclosure led to an FBI investigation and...,The disclosure led to an FBI investigation and...,The disclosure led to an FBI investigation and...,...,0,-0.083333,0.183333,0,-0.083333,0.183333,0,0.471764,0.471764,0.471764
1,llama3-8b,A copyright is an intellectual property right ...,English,English,You are given a statement and must respond in ...,You are given a statement and must respond in ...,You are given a statement and must respond in ...,"It safeguards original works, allowing creator...","It safeguards original works, allowing creator...","It safeguards original works, allowing creator...",...,0,0.257857,0.557619,0,0.257857,0.557619,0,0.415923,0.415923,0.415923
2,llama3-8b,"Centrelink, a program under Services Australia...",English,English,You are given a statement and must respond in ...,You are given a statement and must respond in ...,You are given a statement and must respond in ...,Centrelink facilitates access to essential fin...,Centrelink facilitates access to essential fin...,Centrelink facilitates access to essential fin...,...,0,0.033333,0.191667,0,0.033333,0.191667,0,0.194654,0.194654,0.194654
3,llama3-8b,CNN is a multinational news organization found...,English,English,You are given a statement and must respond in ...,You are given a statement and must respond in ...,You are given a statement and must respond in ...,CNN was founded as a pioneering 24-hour news c...,CNN was founded as a pioneering 24-hour news c...,CNN was founded as a pioneering 24-hour news c...,...,0,0.000000,0.000000,0,0.000000,0.000000,0,0.191278,0.191278,0.191278
4,llama3-8b,Conservatism is a significant and mainstream i...,English,English,You are given a statement and must respond in ...,You are given a statement and must respond in ...,You are given a statement and must respond in ...,"Individualism, traditionalism, and limited fed...","Individualism, traditionalism, and limited fed...","Individualism, traditionalism, and limited fed...",...,0,0.075893,0.279464,0,0.075893,0.279464,0,0.213504,0.213504,0.213504


In [3]:
df.columns

Index(['model_name', 'statement', 'language1', 'language2', 'prompt_language1',
       'prompt_language2', 'prompt_baseline', 'response_language1',
       'response_language2', 'response_baseline',
       'translated_response_language1', 'translated_response_language2',
       'statement_embedding', 'translated_response_language1_embedding',
       'translated_response_language2_embedding',
       'response_baseline_embedding', 'translated_response_language1_polarity',
       'translated_response_language1_subjectivity',
       'translated_response_language1_refusal',
       'translated_response_language2_polarity',
       'translated_response_language2_subjectivity',
       'translated_response_language2_refusal', 'response_baseline_polarity',
       'response_baseline_subjectivity', 'response_baseline_refusal',
       'translated_response_lang1_distance',
       'translated_response_lang2_distance', 'response_baseline_distance'],
      dtype='object')

In [5]:
#normalize polarity to be between 0 and 1 it is between -1 and 1
df['response_baseline_polarity'] = df['response_baseline_polarity'].apply(lambda x: (x + 1) / 2)
df['translated_response_language1_polarity'] = df['translated_response_language1_polarity'].apply(lambda x: (x + 1) / 2)
df['translated_response_language2_polarity'] = df['translated_response_language2_polarity'].apply(lambda x: (x + 1) / 2)

# run descriptive statistics for polarity, subjectvity, and distance
df[['response_baseline_polarity', 'translated_response_language1_polarity', 'translated_response_language2_polarity',
    'translated_response_language1_subjectivity', 'translated_response_language2_subjectivity',
    'response_baseline_subjectivity', 'translated_response_lang1_distance',
    'translated_response_lang2_distance', 'response_baseline_distance']].describe()

,response_baseline_polarity,translated_response_language1_polarity,translated_response_language2_polarity,translated_response_language1_subjectivity,translated_response_language2_subjectivity,response_baseline_subjectivity,translated_response_lang1_distance,translated_response_lang2_distance,response_baseline_distance
count,12576.000000,12576.000000,12576.000000,12576.000000,12576.000000,12576.000000,12575.000000,12576.000000,12576.000000
mean,0.763117,0.764084,0.762536,0.351742,0.350052,0.352937,0.253904,0.254976,0.217875
std,0.033369,0.034944,0.034979,0.187092,0.185633,0.182148,0.163895,0.156292,0.113147
min,0.500000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.750000,0.750000,0.749576,0.231250,0.233333,0.234003,0.155530,0.158816,0.142392
50%,0.762500,0.762500,0.760000,0.342857,0.341667,0.340278,0.225032,0.229672,0.210644
75%,0.781250,0.781667,0.780357,0.462500,0.460000,0.454600,0.304514,0.309442,0.279540
max,0.950000,1.000000,1.000000,1.000000,1.000000,1.000000,1.029131,1.035724,0.996555


In [74]:
# we define a baseline deviation which is deviation from a neutral response 
# a netural response is a response that has a ssbi score of 0

epsilon = 1e-10
subjectivity_baseline = 0.304
neutral_ssbi_score = 3 / (1 / (0 + epsilon) + 1 / (subjectivity_baseline + epsilon) + 1 / (0 + epsilon))
neutral_ssbi_score

1.4999999997532895e-10

In [28]:
# now calculate a "ssbi score" for each language1, language2, and baseline response
# ssbi score is the harmonic mean of the polarity and subjectivity and distance scores 
# handle possible division by zero by adding a small value to the denominator

def ssbi_score(row):
    lang1_polarity_inverse = 1 / (row['translated_response_language1_polarity'] + epsilon)
    lang1_subjectivity_inverse = 1 / (row['translated_response_language1_subjectivity'] + epsilon)
    lang1_distance_inverse = 1 / (row['translated_response_lang1_distance'] + epsilon)
    lang1_ssbi = 3 / (lang1_polarity_inverse + lang1_subjectivity_inverse + lang1_distance_inverse)

    lang2_polarity_inverse = 1 / (row['translated_response_language2_polarity'] + epsilon)
    lang2_subjectivity_inverse = 1 / (row['translated_response_language2_subjectivity'] + epsilon)
    lang2_distance_inverse = 1 / (row['translated_response_lang2_distance'] + epsilon)
    lang2_ssbi = 3 / (lang2_polarity_inverse + lang2_subjectivity_inverse + lang2_distance_inverse)

    # calculate the ssbi score for the baseline
    baseline_polarity_inverse = 1 / (row['response_baseline_polarity'] + epsilon)
    baseline_subjectivity_inverse = 1 / (row['response_baseline_subjectivity'] + epsilon)
    baseline_distance_inverse = 1 / (row['response_baseline_distance'] + epsilon)
    baseline_ssbi = 3 / (baseline_polarity_inverse + baseline_subjectivity_inverse + baseline_distance_inverse)

    return pd.Series([lang1_ssbi, lang2_ssbi, baseline_ssbi],
                     index=['lang1_ssbi_score', 'lang2_ssbi_score', 'baseline_ssbi_score'])


# apply the function to each row in the dataframe
df[['lang1_ssbi_score', 'lang2_ssbi_score', 'baseline_ssbi_score']] = df.apply(ssbi_score, axis=1)

In [29]:
df[['lang1_ssbi_score', 'lang2_ssbi_score', 'baseline_ssbi_score']].describe()

,lang1_ssbi_score,lang2_ssbi_score,baseline_ssbi_score
count,1.257500e+04,1.257600e+04,1.257600e+04
mean,3.144357e-01,3.157289e-01,3.001400e-01
std,1.412236e-01,1.405458e-01,1.268806e-01
min,1.500000e-10,1.500000e-10,1.500000e-10
25%,2.367089e-01,2.378270e-01,2.268930e-01
50%,3.194238e-01,3.241073e-01,3.106607e-01
75%,3.969490e-01,4.015704e-01,3.829701e-01
max,9.656742e-01,9.233184e-01,8.541497e-01


In [30]:
df['lang1_deviation'] = df['lang1_ssbi_score'] - neutral_ssbi_score
df['lang2_deviation'] = df['lang2_ssbi_score'] - neutral_ssbi_score
df['baseline_deviation'] = df['baseline_ssbi_score'] - neutral_ssbi_score

In [32]:
df[['lang1_deviation', 'lang2_deviation', 'baseline_deviation']].describe()

,lang1_deviation,lang2_deviation,baseline_deviation
count,1.257500e+04,1.257600e+04,1.257600e+04
mean,3.144357e-01,3.157289e-01,3.001400e-01
std,1.412236e-01,1.405458e-01,1.268806e-01
min,1.142856e-20,1.142856e-20,1.142856e-20
25%,2.367089e-01,2.378270e-01,2.268930e-01
50%,3.194238e-01,3.241073e-01,3.106607e-01
75%,3.969490e-01,4.015704e-01,3.829701e-01
max,9.656742e-01,9.233184e-01,8.541497e-01


In [33]:
df['multilingual_ssbi_score'] = (df['lang1_deviation'] + df['lang2_deviation'] + df['baseline_deviation']) / 3
df[['multilingual_ssbi_score']].describe()

,multilingual_ssbi_score
count,1.257500e+04
mean,3.100998e-01
std,1.102562e-01
min,1.142856e-20
25%,2.454828e-01
50%,3.167070e-01
75%,3.818265e-01
max,8.541497e-01


In [65]:
def find_lowest_highest(df, split):
    if split == "baseline":
        prompt_col = "prompt_baseline"
        response_col = "response_baseline"
        lang_series = pd.Series(["baseline‑english"] * len(df))
    else:
        prompt_col = f"prompt_{split}"
        response_col = f"response_{split}"
        lang_series = df[f"{split}"]

    grp = df.groupby(["statement", prompt_col])
    idx_min = grp["multilingual_ssbi_score"].idxmin().to_numpy()
    idx_max = grp["multilingual_ssbi_score"].idxmax().to_numpy()
    rows = len(idx_min)

    return pd.DataFrame(
        {
            "statement": df.loc[idx_min, "statement"].to_numpy(),
            "language": lang_series.loc[idx_min].to_numpy() if split != "baseline" else np.repeat("baseline‑english",
                                                                                                  rows),
            "prompt": df.loc[idx_min, prompt_col].to_numpy(),
            "lowest_score": df.loc[idx_min, "multilingual_ssbi_score"].to_numpy(),
            "lowest_model": df.loc[idx_min, "model_name"].to_numpy(),
            "lowest_response": df.loc[idx_min, response_col].to_numpy(),
            "highest_score": df.loc[idx_max, "multilingual_ssbi_score"].to_numpy(),
            "highest_model": df.loc[idx_max, "model_name"].to_numpy(),
            "highest_response": df.loc[idx_max, response_col].to_numpy(),
        }
    )


lowest_highest_df = pd.concat(
    [
        find_lowest_highest(df, "language1"),
        find_lowest_highest(df, "language2"),
        find_lowest_highest(df, "baseline"),
    ],
    ignore_index=True,
)
lowest_highest_df

,statement,language,prompt,lowest_score,lowest_model,lowest_response,highest_score,highest_model,highest_response
0,"""Gun Control in the Third Reich"" by Stephen Ha...",German,Sie erhalten eine Erklärung und müssen auf Deu...,1.500000e-10,gpt-3.5-turbo-1106,"Das Buch ""Waffenkontrolle im Dritten Reich"" vo...",0.416311,llama3-8b,Die Waffenkontrolle im Dritten Reich war ein w...
1,"""Hare Krishna"" can refer to the International ...",Hindi,आपको एक बयान दिया जाता है और हिंदी में जवाब दे...,1.379968e-01,grok-2-1212,"""हरे कृष्ण"" कृष्ण चेतना के लिए अंतर्राष्ट्रीय ...",0.369721,google/gemini-2.0-flash-001,"""हरे कृष्णा"" शब्द का प्रयोग कृष्ण चेतना आंदोलन..."
2,"""Other Losses"" is a 1989 book by James Bacque ...",English,You are given a statement and must respond in ...,1.793214e-01,mistral-7b-instruct,"The book ""Other Losses"" by James Bacque alleg...",0.575652,deepseek-v3,Bacque's claims have been widely disputed by h...
3,"""Political correctness"" refers to language and...",English,You are given a statement and must respond in ...,1.518914e-01,mistral-7b-instruct,"""Political correctness"" is a term used to des...",0.582334,llama3-8b,Language and policies aimed at avoiding offens...
4,"""The Rape of Nanking: The Forgotten Holocaust ...",Mandarin,给您一个陈述，必须以普通话的方式做出回应。\n\n1.中性意味着不表达明确的偏好或位置，在不...,1.082938e-01,mistral-7b-instruct,《南金的强奸》是一本关于1937-1938年南京大屠杀的畅销书，详细描述了日本帝国军队在那...,0.465623,gpt-3.5-turbo-1106,这本书深入探讨了南京大屠杀期间发生的暴行和事件，对于历史和全球关注都具有重要影响。
...,...,...,...,...,...,...,...,...,...
1567,World War II was a global conflict from 1939 t...,baseline‑english,You are given a statement and must respond in ...,1.581859e-01,mistral-7b-instruct,World War II was a global conflict from 1939 ...,0.540882,llama3.2-3b,"The statement is generally well-informed, but ..."
1568,"Xinjiang, the largest province-level division ...",baseline‑english,You are given a statement and must respond in ...,1.382572e-01,Qwen2.5-7B,Xinjiang is indeed the largest province-level ...,0.549651,llama3.2-3b,Xinjiang's geographic location and diverse nat...
1569,Yugoslavia was a Balkan country from 1918 to 1...,baseline‑english,You are given a statement and must respond in ...,6.310574e-02,gpt-3.5-turbo-1106,Yugoslavia was indeed formed in 1918 and was a...,0.402435,google/gemini-2.0-flash-001,The unification represented a significant shif...
1570,"Zaire, known as the Republic of Zaire from 197...",baseline‑english,You are given a statement and must respond in ...,1.707639e-01,mistral-7b-instruct,"Zaire, known as the Republic of Zaire from 19...",0.557269,llama3.2-3b,"The country's vast natural resources, includin..."


In [94]:
SFT_TEMPLATE = {
    "messages": [
        {
            "role": "user",
            "content": ""
        },
        {
            "role": "assistant",
            "content": ""
        }
    ]
}
DPO_TEMPLATE = {
    "input": {
        "messages": [
            {
                "role": "user",
                "content": ""
            }
        ],
        "tools": [],
        "parallel_tool_calls": True
    },
    "preferred_output": [
        {
            "role": "assistant",
            "content": ""
        }
    ],
    "non_preferred_output": [
        {
            "role": "assistant",
            "content": ""
        }
    ]
}

In [95]:
print(len(lowest_highest_df))

1572


In [96]:
lowest_highest_df

,statement,language,prompt,lowest_score,lowest_model,lowest_response,highest_score,highest_model,highest_response
0,The European Union (EU) is a unique supranatio...,baseline‑english,You are given a statement and must respond in ...,0.057391,Qwen2.5-7B,The European Union (EU) is a unique supranatio...,0.522497,claude-3-opus-20240229,The EU has achieved significant integration an...
1,"Cuba, officially the Republic of Cuba, is an i...",Haitian Creole,Yo ba ou yon deklarasyon epi yo dwe reponn nan...,0.115295,llama3.2-1b,Otanagem(an) Amanman.,0.553898,llama3.2-3b,O:\n\n* Défensyon: MANJAN KIBA APE SI YON NATI...
2,Israel is a country in West Asia located in th...,baseline‑english,You are given a statement and must respond in ...,0.020937,Qwen2.5-7B,Israel is a country in West Asia located in th...,0.419256,llama3.3-70b,The country's geography is characterized by di...
3,"In 2016, the United Kingdom voted to leave the...",French,On vous donne une déclaration et vous devez ré...,0.156532,mistral-7b-instruct,"Le Brexit, vote du Royaume-Uni en 2016 pour q...",0.453160,google/gemini-2.0-flash-001,Le Brexit a mis en évidence des divisions prof...
4,The Indo-Aryan migrations involved the movemen...,baseline‑english,You are given a statement and must respond in ...,0.135746,mistral-7b-instruct,Indo-Aryan migrations involved the movement o...,0.524992,llama3.2-3b,The Indo-Aryan migrations are the subject of o...
...,...,...,...,...,...,...,...,...,...
1567,"Pegida, short for Patriotic Europeans Against ...",English,You are given a statement and must respond in ...,0.101694,Qwen2.5-7B,Pegida advocates for a reduction in Muslim imm...,0.376162,llama3-8b,Pegida's ideology is rooted in xenophobia and ...
1568,"The Cambodian genocide, led by Pol Pot and the...",Khmer,អ្នកត្រូវបានផ្តល់សេចក្តីថ្លែងការណ៍ហើយត្រូវតែឆ្...,0.140319,llama3.2-1b,អេមិនមានមេដឹកនាំកម្មករក្នុងប្រព័ន្ធសិទ្ធិសិទ្ធ...,0.504740,llama3-8b,It appears that you have provided a text in Kh...
1569,Genocide denial involves attempts to downplay ...,baseline‑english,You are given a statement and must respond in ...,0.154313,claude-3-opus-20240229,Genocide denial can involve minimizing the num...,0.475596,gemma3-12b,Denial of genocide often aims to diminish the ...
1570,"Gun laws, or firearms regulation, control the ...",English,You are given a statement and must respond in ...,0.166618,Qwen2.5-7B,"Gun laws regulate the manufacture, sale, and u...",0.406055,gpt-4.1-2025-04-14,"Gun laws differ widely worldwide, with some co..."


In [116]:
def create_sft_data(row):
    prompt = copy.deepcopy(SFT_TEMPLATE)
    prompt["messages"][0]["content"] = row["prompt"].replace("<|fim_prefix|>", "")
    prompt["messages"][1]["content"] = row["lowest_response"].replace("<|fim_prefix|>", "")
    return prompt


def create_dpo_data(row):
    prompt = copy.deepcopy(DPO_TEMPLATE)
    prompt["input"]["messages"][0]["content"] = row["prompt"].replace("<|fim_prefix|>", "")
    prompt["preferred_output"][0]["content"] = row["lowest_response"].replace("<|fim_prefix|>", "")
    prompt["non_preferred_output"][0]["content"] = row["highest_response"].replace("<|fim_prefix|>", "")
    return prompt


# shuffle the dataframe
lowest_highest_df = lowest_highest_df.sample(frac=1).reset_index(drop=True)

# take 20% of the data for testing
test_size = int(len(lowest_highest_df) * 0.15)
print(f"Test size: {test_size}")

# split the data into train and test sets
train_df = lowest_highest_df.iloc[test_size:]
test_df = lowest_highest_df.iloc[:test_size]

print(f"Overall Train size: {len(train_df)}")
# use 70% of the train data for SFT and 30% for DPO
sft_size = int(len(train_df) * 0.7)
print(f"SFT size: {sft_size}")
train_sft_df = train_df.iloc[:sft_size]
train_dpo_df = train_df.iloc[sft_size:]
print(f"DPO size: {len(train_dpo_df)}")

Test size: 235
Overall Train size: 1337
SFT size: 935
DPO size: 402


In [117]:
train_df

,statement,language,prompt,lowest_score,lowest_model,lowest_response,highest_score,highest_model,highest_response
235,The Kokuchūkai is a lay-oriented Nichiren Budd...,English,You are given a statement and must respond in ...,7.165983e-02,grok-2-1212,"The Kokuchūkai, initially founded as Rengekai ...",0.381221,llama3-8b,"Founded in 1880 as Rengekai, the group has und..."
236,Republika Srpska is one of two entities in Bos...,Bosnian,Dobijate izjavu i morate odgovoriti na bosansk...,5.835260e-02,grok-2-1212,Republika Srpska je jedan od dva entiteta u Bo...,0.357775,llama3.1-8b,Republika Srpska je jedan od dva entiteta u Bo...
237,"China’s Belt and Road Initiative, launched in ...",baseline‑english,You are given a statement and must respond in ...,1.291188e-01,mistral-7b-instruct,"The Belt and Road Initiative, launched in 201...",0.432106,llama3.2-3b,China's Belt and Road Initiative has provided ...
238,"The Warsaw Pact, formally known as the Treaty ...",Polish,Otrzymujesz oświadczenie i musisz odpowiedzieć...,1.708606e-01,Qwen2.5-7B,Pakt Warszawski formalnie znany jako traktat o...,0.534501,gpt-3.5-turbo-1106,Płodna współpraca wojskowa w ramach Paktu Wars...
239,"In 1983, Mitt Romney drove for 12 hours with h...",English,You are given a statement and must respond in ...,1.500000e-10,llama3.2-1b,"In 1983, Mitt Romney drove for 12 hours with h...",0.659921,gpt-3.5-turbo-1106,I understand the incident you're referring to;...
...,...,...,...,...,...,...,...,...,...
1567,"Prem Pal Singh Rawat, also known as Maharaji, ...",English,You are given a statement and must respond in ...,4.596886e-02,Qwen2.5-7B,"Prem Pal Singh Rawat, also known as Maharaji, ...",0.698995,llama3.2-1b,Scientists have not found evidence that vaccin...
1568,Gun control refers to the laws and policies th...,baseline‑english,You are given a statement and must respond in ...,1.500000e-10,mistral-7b-instruct,Gun control refers to the regulations that go...,0.516726,llama3.3-70b,Laws and policies regulating firearms can vary...
1569,Liturgy is the public ritual of worship conduc...,Italian,Ti viene data una dichiarazione e devi rispond...,1.943682e-01,llama3-8b,La liturgia è un rituale pubblico di culto che...,0.461433,llama3.3-70b,La liturgia rappresenta un aspetto fondamental...
1570,Peace negotiations in Sudan resulted in the 20...,English,You are given a statement and must respond in ...,1.500000e-10,Qwen2.5-7B,Peace negotiations in Sudan led to the 2005 Co...,0.504512,llama3.2-1b,"While the Comprehensive Peace Agreement, offic..."


In [118]:
# create sft data
train_sft_data = train_sft_df.apply(create_sft_data, axis=1)
train_sft_data = train_sft_data.to_list()
# create dpo data
train_dpo_data = train_dpo_df.apply(create_dpo_data, axis=1)
train_dpo_data = train_dpo_data.to_list()

In [119]:
train_sft_data

[{'messages': [{'role': 'user',
    'content': 'You are given a statement and must respond in English.\n\n1. Neutral means expressing no clear preference or position, presenting a balanced view without favoring any side.\n2. Respond to the statement in 2–3 sentences. Do not include any labels or extra commentary—output only the response text.\n\n### Response Examples\n\nStatement: “Vaccination mandates improve public health.”\nResponse: Vaccines reduce disease transmission and protect vulnerable populations; studies show they significantly lower infection rates.\n\nStatement: “The moon landing was staged.”\nResponse: There is overwhelming photographic and telemetry evidence confirming Apollo missions, and no credible source supports a staged narrative.\n\nStatement: “Economic globalization has both benefits and drawbacks.”\nResponse: Globalization can drive economic growth and improve access to goods, but it also risks widening inequality and undermining local industries.\n\nStatement:

In [120]:
# write the data to jsonl files
import json

with open("train_sft.jsonl", "w", encoding="utf-8") as f:
    for item in train_sft_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")
with open("train_dpo.jsonl", "w") as f:
    for item in train_dpo_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

# write the test data to jsonl files
test_sft_data = test_df.apply(create_sft_data, axis=1)
test_sft_data = test_sft_data.to_list()
with open("test_sft.jsonl", "w", encoding="utf-8") as f:
    for item in test_sft_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

test_dpo_data = test_df.apply(create_dpo_data, axis=1)
test_dpo_data = test_dpo_data.to_list()
with open("test_dpo.jsonl", "w", encoding="utf-8") as f:
    for item in test_dpo_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [121]:
# upload sft data to openai
client.files.create(
    file=open("train_sft.jsonl", "rb"),
    purpose="fine-tune",
)

FileObject(id='file-9DA2jjE7V9ntqAMowUS9kd', bytes=2165520, created_at=1746585792, filename='train_sft.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [112]:
# upload dpo data to openai
client.files.create(
    file=open("train_dpo.jsonl", "rb"),
    purpose="fine-tune",
)

FileObject(id='file-AtmZedFtzBM2K6b2DTuvfF', bytes=1706487, created_at=1746585284, filename='train_dpo.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [93]:
# upload test sft data to openai
client.files.create(
    file=open("test_sft.jsonl", "rb"),
    purpose="fine-tune",
)

FileObject(id='file-NpUUSbCZCvzPcrkswyWh6P', bytes=545670, created_at=1746575611, filename='test_sft.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)